In [76]:
import cv2
import numpy as np
import math

data = []
hits = []

image = cv2.imread('./images/targets/img12.jpg')
w = image.shape[1]
h = image.shape[0]

imageO = image.copy()

ratio = h/w
dim = (700, int(700*ratio))
imageO = cv2.resize(imageO, dim, interpolation = cv2.INTER_AREA)
image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

grayC = cv2.cvtColor(imageO,cv2.COLOR_BGR2GRAY)

k, grayC = cv2.threshold(grayC,90,255,cv2.THRESH_BINARY_INV)
edgeC = cv2.Canny(grayC,50,255)


contoursC, hierarchy = cv2.findContours(edgeC, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
arr = []
R = 0
for c in contoursC:
    (x,y,w,h) = cv2.boundingRect(c)
    #cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,255), 1, 8, 0)
    if(w > 200 and w < 400 and h > 200 and h < 400):
        arr.append((x,y,w,h,(w-h)))
arr = sorted(arr, key=lambda a: (-a[3],a[4]))
(x,y,w,h,t) = arr[0]
data.append(w)
R = int((w/2)/7.4)
image = image[y-120:y+h+120,x-120:x+w+120]
(w,h,c) = image.shape
image = cv2.circle(image,(int(w/2),int(h/2)),1,(0,255,0),5,8,0)
data.append((int(w/2),int(h/2)))
lower = np.array([80, 0, 108])
upper = np.array([179, 255, 255])

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower, upper)

kernel1 = np.ones((5,5), np.uint8)

output = cv2.bitwise_and(image,image, mask= mask)
gray = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)
erosion = cv2.erode(gray,kernel1,iterations = 1)
dil = cv2.dilate(erosion,kernel1,iterations = 1)
edge = cv2.Canny(dil,50,255)
contours, hierarchy = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


bg = np.zeros(gray.shape,dtype='uint8')
bg1 = np.zeros(gray.shape,dtype='uint8')
for c in contours:
    (x,y,w,h) = cv2.boundingRect(c)
    for i in range(len(c)):
        cont = c[i][0]
        bg1 = cv2.circle(bg1,(cont[0],cont[1]),R,73,1,8,0)
        bg = cv2.addWeighted(bg,1,bg1,1,0)
        #cv2.imshow('detect les',bg)
        #cv2.waitKey(0)
        bg1 = np.zeros(gray.shape,dtype='uint8')
    k, bg = cv2.threshold(bg, 220, 255, cv2.THRESH_BINARY)
    #dil = cv2.dilate(bg,kernel1,iterations = 2)
    #bg = cv2.erode(dil,kernel1,iterations = 1)
    bg = cv2.medianBlur(bg,7)
    #cv2.imshow('dcted les',bg)
    #cv2.waitKey(0)
    contoures, hierarchyy = cv2.findContours(bg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in contoures:
        M = cv2.moments(c)
        if(M['m00'] != 0):
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            #print(cx)
            #print(cy)
            #if(Pada(x,y,w,h,cx,cy,R+5)):
            if(cx < dil.shape[0] and cy < dil.shape[1] and dil[cy,cx] > 50):
                hits.append((cx,cy))
                #image = cv2.circle(image,(cx,cy),R,(255,0,0),1,8,0)
                #image = cv2.circle(image,(cx,cy),1,(0,0,255),4,8,0)
    bg = np.zeros(gray.shape,dtype='uint8')
data.append(hits)
print(data[0])
segment = data[0]/2/6
for h in data[2]:
    image = cv2.circle(image,h,R,(255,0,0),1,8,0)
    image = cv2.circle(image,h,1,(0,0,255),4,8,0)
    a = h[0] - data[1][0]
    b = h[1] - data[1][1]  
    d = (a*a) + (b*b)
    d = math.sqrt(d)
    print(d)
    print(10-((d-R)/segment))
    cv2.imshow('dcteles',image)
    cv2.waitKey(0)
    
cv2.imshow('dcteles',image)
cv2.waitKey(0)   
cv2.destroyAllWindows()

332
77.80102827083971
7.983095363704589
68.06614430096654
8.334958639724102
41.400483088968905
9.298777719675822
39.824615503479755
9.355736789030852
151.5585695366646
5.31716013722899
